In [14]:
import pandas as pd
import random
import geopandas as gpd
import numpy as np
import os
from matplotlib import pyplot as plt
import pickle
from IPython.display import Image
random.seed(9001)

### Importing the data for North Carolina

In [6]:
os.chdir('/Users/luishonsel/Dropbox/DPI 610/Luis_Code')

#Shapefile for North Carolina
shapefile = gpd.read_file("../NC_VTD/NC_VTD.shp").set_index('VTD')

As part of this project, one my my classmates, Luca Mingardi, prepared an adjacency matrix.  
Such a matrix will be squared of size n, n being the number of VTDs (Voting Districts), and the (i,j) coordinate will be equal to 1 if i-th VTD is neighbour to j-th VTD.  
In this code I will import the Adjacency matrix already done and will not code it.

In [8]:
#importing adjacency matrix
adj_matrix=pd.read_csv('DUMMY_adj_mat.csv').set_index('VTD_Key')


list_VTD=adj_matrix.index

### Preparing some basic statitics that we will use later on for our model

In [9]:
#total state population
state_pop=shapefile.PL10AA_TOT.sum()

#maximum ratio of population per district our algorithm will authorize
max_ratio=state_pop/12
nb_district=13

#population share per state if perfectly distributed
state_avg_pop=state_pop/nb_district

#democratic voters share per state if perfectly distributed
state_avg_dem=shapefile.EL16G_GV_D.sum()/shapefile.PL10VA_TOT.sum()

#african american voters share per state if perfectly distributed
state_avg_black_share=shapefile.BPOP.sum()/state_pop

shapefile['District']=np.nan

### Defining the class object that will represent Districs , VTDs and remaining space

In [10]:
class District:
    
    def __init__(self,VTDs):
        self.composition = VTDs
        self.number_of_vtd = len(VTDs)
        self.pop=shapefile.loc[VTDs].PL10AA_TOT.sum()
        self.voting_pop=shapefile.loc[VTDs].PL10VA_TOT.sum()
        self.pop_share=self.pop/state_pop
        self.dem_pop=shapefile.loc[self.composition].EL16G_GV_D.sum()
        self.dem_share=self.dem_pop/self.voting_pop
        self.black_pop=shapefile.loc[VTDs].BPOP.sum()
        self.black_share=shapefile.loc[VTDs].BPOP.sum()/self.pop
        self.neighbors=remove(adj_matrix.index[adj_matrix.loc[VTDs].sum()>0],VTDs)
        self.constraint=True
        
    def add(self,new_VTDs):
        self.composition=self.composition+new_VTDs
        self.pop+=shapefile.loc[new_VTDs].PL10AA_TOT.sum()
        self.voting_pop+=shapefile.loc[new_VTDs].PL10VA_TOT.sum()
        self.dem_pop=shapefile.loc[self.composition].EL16G_GV_D.sum()
        self.dem_share=self.dem_pop/self.voting_pop
        self.black_share=shapefile.loc[self.composition].BPOP.sum()/self.pop
        self.neighbors=remove(adj_matrix.index[adj_matrix.loc[self.composition].sum()>0],self.composition)
        self.constraint=self.pop<=max_ratio
        self.pop_share=self.pop/state_pop
        self.black_pop+=shapefile.loc[new_VTDs].BPOP.sum()
        self.number_of_vtd += len(new_VTDs)
        #rahouter des assert partout
    def delete(self,del_VTDs):
        self.composition=remove(self.composition,del_VTDs)
        self.pop-=shapefile.loc[del_VTDs].PL10AA_TOT.sum()
        self.voting_pop-=shapefile.loc[del_VTDs].PL10VA_TOT.sum()
        self.dem_pop=shapefile.loc[self.composition].EL16G_GV_D.sum()
        self.dem_share=self.dem_pop/self.voting_pop
        self.black_share=shapefile.loc[self.composition].BPOP.sum()/self.pop
        self.neighbors=remove(adj_matrix.index[adj_matrix.loc[self.composition].sum()>0],self.composition)
        self.constraint=self.pop<=max_ratio
        self.pop_share=self.pop/state_pop
        self.number_of_vtd -= len(del_VTDs)
        self.black_pop-=shapefile.loc[del_VTDs].BPOP.sum()
        
        
class VTD:
    
    def __init__(self,vtd):
        self.name = vtd
        self.pop=shapefile.loc[vtd].PL10AA_TOT.sum()
        self.voting_pop=shapefile.loc[vtd].PL10VA_TOT.sum()
        self.dem_pop=shapefile.loc[vtd].EL16G_GV_D.sum()
        self.dem_share=self.dem_pop/self.voting_pop
        self.black_share=shapefile.loc[vtd].BPOP.sum()/self.pop
        self.black_pop=shapefile.loc[vtd].BPOP.sum()
        self.neighbors=adj_matrix.index[adj_matrix.loc[vtd]>0]
        self.distric=np.nan
    
class Remaining_Space:
    
    def __init__(self,starting_points):
        self.available_points=remove(list_VTD,starting_points)
        
    def remove(self,point):
        self.available_points=remove(self.available_points,point)
        
    def add(self,point):
        self.available_points+=point

### Now we create some useful function that will help us throughout the algorithm

In [12]:
def remove(list1,list2):
    return [i for i in list1 if i not in list2]

def intersection(list1,list2):
    return [i for i in list1 if i in list2]
      
def status(clusters):
        return sum([i.constraint for i in clusters])
    
#evaluates the score for a certain distribution in one district        
def cost_function_eval(vtd,district,weight_pop,weight_race,weight_vote):
    
    #difference in population repartition
    tot_pop=district.pop+VTD(vtd).pop
    pop_share=tot_pop/state_pop
    avg_diff_pop=abs(pop_share-state_avg_pop)/state_avg_pop
    
    #difference in black population reparition
    black_pop=district.black_pop+VTD(vtd).black_pop
    black_pop_share=black_pop/tot_pop
    avg_diff_black=abs(black_pop_share-state_avg_black_share)
    
    #difference in dem repartition
    dem_pop=district.dem_pop+VTD(vtd).dem_pop
    dem_pop_share=dem_pop/tot_pop
    avg_diff_dem=abs(dem_pop_share-state_avg_dem)
    
    return(weight_pop*avg_diff_pop+weight_race*avg_diff_black+weight_vote*avg_diff_dem)
    
    
#evaluates the score for a certain distribution in all districts        
def cost_function_final(districts,weight_pop,weight_race,weight_vote):
    cost=0
    for dist in districts:
        avg_diff_pop=abs(dist.pop-state_avg_pop)/state_avg_pop
        avg_diff_black=abs(dist.black_share-state_avg_black_share)
        avg_diff_dem=abs(dist.dem_share-state_avg_dem)
        cost+=weight_pop*avg_diff_pop+weight_race*avg_diff_black+weight_vote*avg_diff_dem
    return(cost)
    
    
#indicates what would be the best VTD as a next step
def best_vtd(district,points,weight_pop,weight_race,weight_vote):
    cost=[]
    for vtd in points:
        cost.append(cost_function_eval(vtd,district,weight_pop,weight_race,weight_vote))
    return(district.neighbors[np.argmin(cost)])    

#useful function for printing
def print_clusters(clusters):
    for i in range(len(clusters)):
        shapefile.loc[clusters[i].composition,'District']=i
    return(shapefile.plot(column='District',figsize=(20,40)))  

#find all parents of a VTD
def get_parent_cluster(districts,vtd):
    c=0
    for i in range(len(districts)):
        if vtd in districts[i].composition:
            c=i
    return c
            
#can our district lose this VTD to improve?
def is_expendable(district,vtd):
    neighbors=VTD(vtd).neighbors
    neighbors_in_cluster=[i for i in neighbors if i in district.composition]
    list1=VTD(neighbors_in_cluster[0]).neighbors
    for vtd in neighbors_in_cluster:
        list1=intersection(list1,VTD(vtd).neighbors)
    a=0
    for element in list1:
        if element in district.composition:
            a+=1
    return a>=2

# Methods Used

This method uses a classical heuristic algorithm widely used in optimization, the 2-OPT algorithm. Basically, this method starts from one feasible solution and explores all the neighboring ones that improve our cost function. Before going in detail into the 2-OPT algorithm, let’s first see how we generate feasible solutions to start with.

## Algorithm: Random Start

The Random start algorithm assign 13 random points to 13 distinct districts. At each iteration, each district gets added a new point, selected randomly among its neighboring available point. The algorithm stops when there are no available points left.
0. Let S be the set of available point. S is comprised of all the states VTDs for now. We randomly select 13 points among S that will define the 13 distinct clusters.
1. Remove those 13 points from S
2. While S is not empty :  
(a) For each cluster look at the neighboring points still in S  
(b) Randomly select one of these points and add it to the cluster  
(c) Remove this point from S

The following figures give you the kind of redistricting you obtain though this method, through different iterations

<img src="Picture1.png">
<img src="Picture2.png">
<img src="Picture3.png">
<img src="Picture4.png">

In [13]:
#Code to create the random start we need before the optimization    
def random_init(length=1000,nb_of_districts=13,weight_pop=1,weight_race=1,weight_vote=1):
    random_points=random.choices(list_VTD, k=nb_of_districts)
    space=Remaining_Space(random_points)
    clusters=[District([i]) for i in random_points]
    state=status(clusters)
    t=0
    while state>0 and t<10000:
        if len(space.available_points)!=0:
            for clust in clusters:
                try:
                    possible_points=intersection(clust.neighbors,space.available_points)
                    point=random.choices(possible_points, k=1)
                    clust.add(point)
                    space.remove(point)
                except:
                    pass
                state=status(clusters)
                
        
        t+=1
    return(clusters,cost_function_final(clusters,weight_pop,weight_race,weight_vote))

## Algorithm: 2-OPT

Once we have our feasible solutions to start, we can use the 2-OPT algorithm. The main idea is to explore all neighbouring solutions of our current solution. A neighbor will be a solution that only differs by one point. If the neighbor has a better score, it will be your new current solution. The algorithm runs until there are no more improvements possible.
0. Let V be our current solution
1. While there are changes in the following loop  
(a) Look at every neighbor N of V  
(b) If N has a better score than V, V :=N  
(c) Otherwise move to another neighbouring solution

<img src="example.png">

You use this method on top of the random start. it provides the following kind of solution:
<img src="opt.png">


Evaluation
This method presents different advantages, among which:  

- Guaranteed contiguity: given the way the random start algorithm works, adding VTDs to clusters one at a time, and also given the way the 2-OPT algorithm works, we are assured that there will always be a path between any two points inside a cluster.  
- The optimization part allows this method to provide better score than Weighted K-means or random start methods.
- This model can provide different outputs depending on the metric we wan to prioritize.

On the other hands, some of the drawbacks are:  
- Locally optimal solution: Given that the 2-OPT method only examines the neigh- bours of a current solution, we can easily get stuck in a locally optimal solution and be far from a globally optimal one.  
- Because of the way the districts are built, this methods does not perform as well as Weighted K means for the compactness score.  
• This method is highly dependent on the starting point and we observe a high variance at each new output of the algorithm.

In [15]:
def optimization_1(districts,weight_pop=1,weight_race=1,weight_vote=1,tim=10):
    curr_cost=cost_function_final(districts,weight_pop,weight_race,weight_vote)
    for t in range(tim):
        change=0
        print('Completion status of main loop: ',t+1,' out of ',tim)
        for i in range(len(districts)):
            print('Completion status of sub loop: ',i+1,' out of ',len(districts))
            to_visit=districts[i].neighbors  
            for vtd in to_visit:
                parent_cluster=get_parent_cluster(districts,vtd)
                if is_expendable(districts[parent_cluster],vtd):
                    old_cost=cost_function_final([districts[i],districts[parent_cluster]],weight_pop,weight_race,weight_vote)
                    
                    #cluster A
                    cluster_A=District(districts[i].composition)
                    cluster_A.add([vtd])
                    
                    #cluster B
                    cluster_B=District(districts[parent_cluster].composition)
                    cluster_B.delete([vtd])
                    
                    
                    new_cost=cost_function_final([cluster_A,cluster_B],weight_pop,weight_race,weight_vote)
                    if new_cost<old_cost:
                        print('success')
                        change+=1
                        districts[i].add([vtd])
                        districts[parent_cluster].delete([vtd])
        if change==0:
            break
    new_cost=cost_function_final(districts,weight_pop,weight_race,weight_vote)
    return(districts,print_clusters(districts),print('The score has been improved by',curr_cost-new_cost))
    
def optimization_2(districts,weight_pop=1,weight_race=1,weight_vote=1,tim=10):
    curr_cost=cost_function_final(districts,weight_pop,weight_race,weight_vote)
    for t in range(tim):
        print('Completion status: ',t/tim)
        change=0
        for i in range(len(districts)):
            to_visit=districts[i].neighbors  
            for vtd in to_visit:
                parent_cluster=get_parent_cluster(districts,vtd)
                if is_expendable(districts[parent_cluster],vtd):
                    parent_cluster=get_parent_cluster(districts,vtd)
                    old_cost=cost_function_final([districts[i],districts[parent_cluster]],weight_pop,weight_race,weight_vote)
                    
                    #cluster A
                    cluster_A=District(districts[i].composition)
                    cluster_A.add([vtd])
                    
                    #cluster B
                    cluster_B=District(districts[parent_cluster].composition)
                    cluster_B.delete([vtd])
                    
                    
                    new_cost=cost_function_final([cluster_A,cluster_B],weight_pop,weight_race,weight_vote)
                    if new_cost<old_cost:
                        change+=1
                        districts[i].add([vtd])
                        districts[parent_cluster].delete([vtd])
                    else:
                        new_neighbors=VTD(vtd).neighbors
                        for vtd_new_neighbors in new_neighbors:
                            if vtd_new_neighbors in cluster_A.composition:
                                pass
                            else:
                                parent_cluster_new=get_parent_cluster(districts,vtd_new_neighbors)
                                if parent_cluster_new!=parent_cluster:
                                    old_cost=cost_function_final([cluster_A,cluster_B,districts[parent_cluster_new]],weight_pop,weight_race,weight_vote)
                                    
                                    #cluster A
                                    cluster_A_prime=District(cluster_A.composition)
                                    cluster_A_prime.add([vtd_new_neighbors])
                                    
                                    #Cluster C
                                    cluster_C=District(districts[parent_cluster_new].composition)
                                    cluster_C.delete([vtd_new_neighbors])
                                    
                                    new_cost=cost_function_final([cluster_A_prime,cluster_B,cluster_C],weight_pop,weight_race,weight_vote)
                                    
                                    if new_cost<old_cost:
                                        change+=1
                                        districts[i].add([vtd])
                                        districts[i].add([vtd_new_neighbors])
                                        districts[parent_cluster].delete([vtd])
                                        districts[parent_cluster_new].delete([vtd_new_neighbors])
                                else:
                                    old_cost=cost_function_final([cluster_A,cluster_B],weight_pop,weight_race,weight_vote)
                                                                        #cluster A
                                    cluster_A_prime=District(cluster_A.composition)
                                    cluster_A_prime.add([vtd_new_neighbors])
                                    
                                    #Cluster C
                                    cluster_B_prime=District(cluster_B.composition)
                                    cluster_B_prime.delete([vtd_new_neighbors])
                                    
                                    new_cost=cost_function_final([cluster_A_prime,cluster_B_prime],weight_pop,weight_race,weight_vote)
                                    if new_cost<old_cost:
                                        change+=1
                                        districts[i].add([vtd])
                                        districts[i].add([vtd_new_neighbors])
                                        districts[parent_cluster].delete([vtd])
                                        districts[parent_cluster].delete([vtd_new_neighbors])
                                    
                    

                                
                                
                        
        if change==0:
            break
        new_cost=cost_function_final(districts,weight_pop,weight_race,weight_vote)
        return(districts,print_clusters(districts),print('The score has been improved by',curr_cost-new_cost))       
            


## Simulated annealing

As we have seen previously, one of the main drawbacks of the 2-OPT method is that it is very locally optimal.   
While the 2-OPT algorithm only accepts points that lower the objective function, the simulated annealing algorithm accepts not only new points that lower the objective, but also, with a certain probability, points that raise the objective. By doing this, the algorithm avoids being trapped in local minima, and is able to explore globally for more possible solutions. The probability is calculated using a temperature parameter, that will decrease through iterations, reducing the extent of the search and thus assuring that the algorithm will reach a local optimum at some point.

__Algorithm__
The simulated annealing method needs a feasible solution has an input. We use the random start algorithm to generate it.  
__Input:__ Feasible solution V, number of iterations tmax, initial temperature T0, temperature parameter α.
0. Let V be our current solution 
1. For t ranging from 1 to tmax:  
(a) Look at every neighbor N of V  
(b) If N has a better score than V,V :=N  
(c) Otherwise, if we note C(V) and C(N) the costs of solutions V and N, do V := N with a probability
e−(C(V )−C(N))/T(t) (1) where T (t) is the temperature at period t defined as T (t) = T0 ∗ αt  




The results provided are as follows in this picture:
<img src="sim_anne.png">

In [16]:
def temp_log(t_0,t):
    return(t_0/np.log(t))
 
def temp_exp(t_0,t,alpha):
    return(t_0*(alpha**t))

def sim_annealing(districts,method,alpha=0.1,T_0=1,weight_pop=1,weight_race=1,weight_vote=1,tim=100):
    curr_cost=cost_function_final(districts,weight_pop,weight_race,weight_vote)
    cost=[curr_cost]
    for t in range(1,tim):
        print('Completion status of main loop: ',t+1,' out of ',tim)
        i=random.randint(1,len(districts)-1)
        print(i)
        to_visit=districts[i].neighbors  
        vtd=random.choice(to_visit)
        parent_cluster=get_parent_cluster(districts,vtd)
        
        if is_expendable(districts[parent_cluster],vtd):
            old_cost=cost_function_final([districts[i],districts[parent_cluster]],weight_pop,weight_race,weight_vote)
            
            #cluster A
            cluster_A=District(districts[i].composition)
            cluster_A.add([vtd])
                    
            #cluster B
            cluster_B=District(districts[parent_cluster].composition)
            cluster_B.delete([vtd])
            
            
            new_cost=cost_function_final([cluster_A,cluster_B],weight_pop,weight_race,weight_vote)
            
            if new_cost<old_cost:
                print('normal change')
                districts[i].add([vtd])
                districts[parent_cluster].delete([vtd])
            else:
                if method=='log':
                    temp=temp_log(T_0,t)
                else:
                    temp=temp_exp(T_0,t,alpha)
                probability=np.exp((old_cost-new_cost)/temp)
                if random.random()<probability:
                    print('probabilistic change')
                    districts[i].add([vtd])
                    districts[parent_cluster].delete([vtd])
        cost.append(cost_function_final(districts,weight_pop,weight_race,weight_vote))
    new_cost=cost_function_final(districts,weight_pop,weight_race,weight_vote)
    return(districts,print_clusters(districts),print('The score has been improved by',curr_cost-new_cost),cost)

#score to ass the convexity of a solution
def convexity_score(shapefile, cluster_results):
    for i in range(len(cluster_results)):
        shapefile.loc[cluster_results[i].composition,'District']=i
    score = 0
    clusters = shapefile['District'].unique().tolist()
    for cluster_id in clusters:
        cluster = shapefile[shapefile['District'] == cluster_id]
        union = cluster.unary_union
        conv_hull = union.convex_hull
        s = union.area / conv_hull.area
        score += s    
    return score / len(clusters)
            
            
       


This notebook is aimed at providing the code and some explanations, for more details about the project please refer to the written report by Alexandru Socolov, Luca Mingardi and I on my github.